#### Notebooks
|  | Name | Duration* |
| ------ | ------ | ------ |
| 1. | [Introduction](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/pywapor_101.ipynb) | 10 + 30 |
| 2. | [Levels](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/levels.ipynb) | 10 + 120 |
| 3. | [Composites](https://colab.research.google.com/github/bertcoerver/pywapor_notebooks/blob/main/composites.ipynb) | 10 + 10 |
| 4. | **pyWaPOR vs. WaPOR** | 10 + 30 |
| 5. | Soil Saturation | > Coming Soon < |

\* Estimation of the time required in minutes, as in "active" + "download time". 

\* Also note that running the different notebooks on the same instance (or your own computer), will save you a lot of "download time" because the downloaded data can be reused between the notebooks.

# pyWaPOR vs. WaPOR

In this notebook we'll make an comparison between dekadal Evapotranspiration data from WaPOR and from pyWaPOR.

First we install pywapor, in case it's not installed yet.

In [ ]:
!pip install pywapor==2.3.2 --quiet

Next we'll define a couple of basic variables, which were explained in the introduction notebook.

In [ ]:
import pywapor

project_folder = r"/Users/hmcoerver/On My Mac/pyWAPOR/example_data"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
startdate = "2021-07-01"
enddate = "2021-07-11"

Then we run `pre_et_look`, to generate an ET map for the first dekad of July 2021. If you already ran another notebook on this instance, you should already have all the data in your RAW folder and this should take only a few minutes.

In [ ]:
pre_files = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim)

Then we run `et_look`. In order to get a good match with WaPOR we'll adjust some of the default constants used by pyWaPOR. The WaPOR dataset is generated with different values for these constants per region, see the WaPOR documentation for more information.

In [ ]:
input_data = {
                "lw_offset": -4.7,
                "lw_slope":  6.94,
                "r0_bare":   0.2650,
                "r0_full":   0.2540,
                "rn_offset": -210.774,
                "rn_slope":  1.394, 
                "t_opt":     23.097,
                "vpd_slope": -0.3,
}

pywapor_files = pywapor.et_look.main(project_folder, startdate, input_data = input_data)

pywapor_file = pywapor_files["et_24_mm"]

Now that we have the pyWaPOR ET map, we still need the WaPOR map in order to make a comparison. We can download it like this.

In [ ]:
import os

raw_folder = os.path.join(project_folder, "RAW")
dl_args = (raw_folder, latlim, lonlim, startdate, enddate, "L1_AETI_D")

wapor_file = pywapor.collect.WAPOR.Get_Layer(*dl_args)[0]

For the rest of this notebook we'll work on comparing the values within these two geoTIFFs.

In [ ]:
print(f"wapor_file  : {os.path.split(wapor_file)[-1]}")
print(f"pywapor_file: {os.path.split(pywapor_file)[-1]}")

We start by opening the data and comparing some basic information. From the geotransforms of the geoTIFFs we can see that the pixel sizes are similar, but not identical. Furthermore, the number of pixels is unequal as well.

In [ ]:
wapor_data = pywapor.general.processing_functions.open_as_array(wapor_file)
wapor_geot = pywapor.general.processing_functions.get_geoinfo(wapor_file)[0]

pywapor_data = pywapor.general.processing_functions.open_as_array(pywapor_file)
pywapor_geot = pywapor.general.processing_functions.get_geoinfo(pywapor_file)[0]

print(f"pixel_size pywapor  : {pywapor_geot[1]:.4f}")
print(f"pixel_size wapor    : {wapor_geot[1]:.4f}")
print(f"array_shape pywapor : {pywapor_data.shape}")
print(f"array_shape wapor   : {wapor_data.shape}")

We'll have to do something about these differences, but let's first create some maps to get a better idea of what we are dealing with.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(1)
fig.clf()
fig.set_size_inches(9.69, 5.27)
axs = fig.subplots(1, 2, sharex=False, sharey=False).flatten()

pywapor.post_et_look.plot_img(axs[0], pywapor_data, "pywapor", "ET [mm/day]")
pywapor.post_et_look.plot_img(axs[1], wapor_data, "wapor", "ET [mm/day]")

You'll see that the spatial extent of the WaPOR geoTIFF is larger than pyWaPORs, e.g. clearly a larger part of the Nile delta is visible on the WaPOR map. This shouldn't be a surprise though, as we've just seen that (1) the WaPOR pixels are larger and (2) the WaPOR geoTIFF has more pixels.

Nevertheless, in order to really compare the two files, we'll have to reproject one of them to match all the pixels with eachother. We'll reproject the WaPOR file to match with the pyWaPOR file using a nearest-neighbor interpolation.

In [ ]:
ds = pywapor.general.processing_functions.reproject_dataset_example(wapor_file, pywapor_file)

wapor_data = pywapor.general.processing_functions.open_as_array(ds)
wapor_geot = pywapor.general.processing_functions.get_geoinfo(ds)[0]

print(f"pixel_size pywapor  : {pywapor_geot[1]:.4f}")
print(f"pixel_size wapor    : {wapor_geot[1]:.4f}")
print(f"array_shape pywapor : {pywapor_data.shape}")
print(f"array_shape wapor   : {wapor_data.shape}")

Now the pixels have the same size and the arrays have the same shape (i.e. the same amount of pixels).

In the figure we've created earlier, you might also have noticed that the ET values on the WaPOR map were extremely high compared to the pyWaPOR map. As you can read in the WaPOR documentation, we need to apply a scale-factor of 0.1 to the pixel values to get the correct values.

In [ ]:
wapor_data *= 0.1

Now that we have reprojected the data and applied the scale-factor, lets plot the maps again.

In [ ]:
fig = plt.figure(2)
fig.clf()
fig.set_size_inches(9.69, 5.27)
axs = fig.subplots(1, 2, sharex=False, sharey=False).flatten()

pywapor.post_et_look.plot_img(axs[0], pywapor_data, "pywapor", "ET [mm/day]")
pywapor.post_et_look.plot_img(axs[1], wapor_data, "wapor", "ET [mm/day]")

You'll see that the spatial extent of the two maps and the range of the ET values is similar now. The pyWaPOR file contains some pixels without any data though (the white patches). To make a fair comparison, we'll mask those pixels in the WaPOR dataset as well. 

In [ ]:
import numpy as np

mask = np.any([np.isnan(pywapor_data), np.isnan(wapor_data)], axis = 0)
pywapor_data[mask] = np.nan
wapor_data[mask] = np.nan

pywapor_data_1d = pywapor_data[~mask].flatten()
wapor_data_1d = wapor_data[~mask].flatten()

print(f"array_shape pywapor : {pywapor_data_1d.shape}")
print(f"array_shape wapor   : {wapor_data_1d.shape}")

Now we have 207236 values per dataset to compare with eachother.

Let's plot our maps again, this time fixing the colorbars of the two maps to be identical.

In [ ]:
fig = plt.figure(3)
fig.clf()
fig.set_size_inches(9.69, 5.27)
axs = fig.subplots(1, 2, sharex=False, sharey=False).flatten()

pywapor.post_et_look.plot_img(axs[0], pywapor_data, "pywapor", "ET [mm/day]", cb_limits = (0.0, 8.0))
pywapor.post_et_look.plot_img(axs[1], wapor_data, "wapor", "ET [mm/day]", cb_limits = (0.0, 8.0))

The maps look very similar, although it looks like the WaPOR values are slightly higher. we can also calculate some statistics now.

In [ ]:
r = pywapor.post_et_look.calc_pearson_correlation([pywapor_data_1d, wapor_data_1d])[0]
nse = pywapor.post_et_look.calc_nash_sutcliffe([pywapor_data_1d, wapor_data_1d])[0]
rmse = pywapor.post_et_look.calc_rmse([pywapor_data_1d, wapor_data_1d])[0]

print(f"r   : {r:.3f}")
print(f"nse : {nse:.3f}")
print(f"rmse: {rmse:.3f} [mm/day]")

And create a scatterplot.

In [ ]:
fig = plt.figure(4)
fig.clf()
ax = fig.gca()

pywapor.post_et_look.plot_hexbin(ax, [pywapor_data_1d, wapor_data_1d], 
                                 xlabel = "ET (pywapor) [mm/day]", 
                                 ylabel = "ET (wapor) [mm/day]",
                                 title = f"r: {r:.3f}, nse : {nse:.3f}, rmse: {rmse:.3f} [mm/day]")

title = fig.suptitle(f"{startdate} - {enddate}")

Although there is some variance, the two maps are very similar. `et_look` is identical to the model used to create the WaPOR data, but `pre_et_look` is not. This variance can thus mainly be explained by differences in the preprocessing of the `et_look` input data and minor differences in the `et_look` model setup (such as the differing constants we saw before).